In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
gdrive_path = "/content/gdrive/MyDrive/<<where ever you want your output>>"

Mounted at /content/gdrive/


In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

!pip install --quiet huggingface-hub evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!nvidia-smi

Tue Nov  7 05:40:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!pip install --quiet pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U datasets evaluate #scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.01, shuffle=False)
train_dataset = dataset["train"]
test_dataset = dataset["test"].train_test_split(test_size=0.5, shuffle=False)
eval_dataset = test_dataset["test"]
test_dataset = test_dataset["train"]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [ ]:
import json
import random
print(train_dataset)
print(eval_dataset)
print(test_dataset)
print(json.dumps(eval_dataset[random.randrange(len(eval_dataset))], indent=2))
print(eval_dataset[random.randrange(len(eval_dataset))].keys())

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 180993
})
Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 915
})
Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 914
})
{
  "id": "bf1fc3d4-98fd-4b84-b4c4-7568f41bbb02",
  "question": "Left Umbilical vein becomes?",
  "opa": "Ligamentum teres",
  "opb": "Ligamentum venosum",
  "opc": "Medial umbilical ligament",
  "opd": "Ligamentum aeriosum",
  "cop": 0,
  "choice_type": "single",
  "exp": "Ligamentum teres",
  "subject_name": "Anatomy",
  "topic_name": null
}
dict_keys(['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'])


In [ ]:
def generate_question(data_point):
    question = f""" {data_point['question']} [0] {data_point['opa']} [1] {data_point['opb']} [2] {data_point['opc']} [3] {data_point['opd']}
    """
    return question

def generate_prompt(data_point):
    full_prompt=f"""You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.\n### Question: {generate_question(data_point)}\n### Answer: """

    if data_point["cop"] != "" and data_point["exp"] != "":
        full_prompt= full_prompt + f"""{data_point["cop"]}\n### Explanation: {data_point["exp"]}"""

    return full_prompt

generate_prompt(eval_dataset[random.randrange(len(eval_dataset))])

'You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.\n### Question:  Turban epiglottitis is a clinical finding in - [0] Tubercular laryngitis [1] Tubercular pharyngitis [2] Polypoid degeneration of vocal cord [3] Subglottic hemangioma\n    \n### Answer: 0\n### Explanation: Ans. is 4a\' i.e.. Tubercular laryngitis Laryngeal examination in TB lary ngitiso Hyperaemia of the vocal cord in its whole extent or confined to posterior part with impairment of adduction is the first sign.o Swelling in the interarytenoid region giving a mammilated appearance,o Ulceration of vocal cord giving mouse-nibbled appearance.o Superficial ragged ulceration on the arytenoids and interarytenoid region,o Granulation tissue in interarvtenoid region or vocal process of arytenoid,o Pseudoedema of the epiglottis \'Turban epiglottis",o Swelling of ventricular bands and aryepiglottic folds,o Marked pallor of surrounding mucosa.'

In [ ]:
print_gpu_utilization()

GPU memory occupied: 233 MB.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    model_max_length=4096,
    device_map="cuda:0",
    add_eos_token=False)

tokenizer.pad_token = tokenizer.eos_token
print_gpu_utilization()

GPU memory occupied: 11117 MB.


In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

def tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    return tokenize(full_prompt)

def generate_tokens(m, prompt, max_new_tokens=32):
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    m.eval()
    with torch.no_grad():
        return tokenizer.decode(m.generate(**model_input, max_new_tokens=max_new_tokens, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.1, eos_token_id=tokenizer.eos_token_id)[0].cuda())

In [ ]:
sample = test_dataset[0].copy()
print(f"answer {sample['cop']}")
print("----------------")
sample['cop'] = ''
sample['exp'] = ''
print(generate_tokens(base_model, generate_prompt(sample), 64))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


answer 0
----------------
<s> You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.
### Question:  In aluminum phosphide poisoning, which is NOT true: AIIMS 10; Punjab 11 [0] Accumulation of acetylcholine at NM junction [1] Cytochrome oxidase inhibition [2] Phosphene formation [3] Metabolic acidosis
    
### Answer: </s>й

### Explanation:

Aluminum phosphide is a pesticide. It is used to kill rats and insects. It is also used to kill mice. It is also used to kill cockroaches. It is also used to kill bed bugs. It is also


In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True,
    use_auth_token=True)


new_model = "MedMistral-7B"
hf_model = PeftModel.from_pretrained(base_model, f"matthewchung74/{new_model}",
    device_map="cuda:0",
    trust_remote_code=True,
    use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
sample = test_dataset[1].copy()
print(f"answer {sample['cop']}")
print("----------------")
sample['cop'] = ''
sample['exp'] = ''
print(generate_tokens(hf_model, generate_prompt(sample), 64))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer 0
----------------
<s> You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.
### Question:  Primary cause of bleeding disorder in liver damage is due to [0] decreased level of prothrombin [1] Lack of vitamin K [2] Platelet deficiency [3] Lack of vitamin B
    
### Answer: 0
### Explanation: None of the above is the primary cause of bleeding disorder in liver damage. The primary cause of bleeding disorder in liver damage is due to decreased level of prothrombin. The liver is the main site of synthesis of prothrombin. The liver is the main site


In [ ]:
def parse_answer(response):
    s = response.split("### Answer:")[1]
    s = s.split("### Explanation")[0].strip()[0]
    return int(s)

In [ ]:
refs = []
predictions = []
for s in test_dataset:
    ref = s['cop']
    refs.append(ref)
    s['cop'] = ''
    s['exp'] = ''
    response = generate_tokens(hf_model, generate_prompt(s), 64)
    predict = parse_answer(response)
    predictions.append(predict)
    print(f"{ref} {predict}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

3 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 3
0 3


In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")
accuracy.add_batch(references=refs, predictions=predictions)
accuracy.compute()

{'accuracy': 0.6663019693654267}

In [ ]:
refs = []
predictions = []
for s in eval_dataset:
    ref = s['cop']
    refs.append(ref)
    s['cop'] = ''
    s['exp'] = ''
    response = generate_tokens(hf_model, generate_prompt(s), 64)
    predict = parse_answer(response)
    predictions.append(predict)
    print(f"{ref} {predict}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

2 3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1 1
3 1


In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")
accuracy.add_batch(references=refs, predictions=predictions)
accuracy.compute()

{'accuracy': 0.6502732240437158}

In [ ]:
from google.colab import runtime
runtime.unassign()